In [1]:
from google.colab import drive
drive.mount("/content/drive")
modelPath = "/content/drive/My Drive/ambulance_prediction"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from imutils import paths
import os
import numpy as np
from keras import applications

Using TensorFlow backend.


In [3]:
img_width, img_height = 150, 150

train_data_dir = "/content/drive/My Drive/ambulance_prediction/data/train"
validation_data_dir = "/content/drive/My Drive/ambulance_prediction/data/validation"
top_model_weights_path = "/content/drive/My Drive/ambulance_prediction/bottleneck_fc_model.h5"

bottleneck_feat_train_file = os.path.join("/content/drive/My Drive/ambulance_prediction", 'bottleneck_features_train.npy')
bottleneck_feat_valid_file = os.path.join("/content/drive/My Drive/ambulance_prediction", 'bottleneck_features_validation.npy')

nb_train_samples = len(list(paths.list_images(train_data_dir)))
nb_validation_samples = len(list(paths.list_images(validation_data_dir)))
epochs = 50
batch_size = 16

In [20]:
def save_bottlebeck_features():
  datagen = ImageDataGenerator(rescale=1. / 255)
  # build the VGG16 network
  model = applications.VGG16(include_top=False, weights='imagenet')

  generator = datagen.flow_from_directory(
      train_data_dir,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode=None,
      shuffle=False)
  predict_size_train = int(math.ceil(nb_train_samples / batch_size))
  bottleneck_features_train = model.predict_generator(generator, predict_size_train)
  np.save(open(bottleneck_feat_train_file, 'wb'), bottleneck_features_train)

  generator = datagen.flow_from_directory(
      validation_data_dir,
      target_size=(img_width, img_height),
      batch_size=batch_size,
      class_mode=None,
      shuffle=False)
  predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
  bottleneck_features_validation = model.predict_generator(generator, predict_size_validation)
  np.save(open(bottleneck_feat_valid_file, 'wb'), bottleneck_features_validation)

In [21]:
save_bottlebeck_features()

Found 1106 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Found 294 images belonging to 2 classes.


In [22]:
def train_top_model():
  train_data = np.load(bottleneck_feat_train_file)
  train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

  validation_data = np.load(bottleneck_feat_valid_file)
  validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

  model = Sequential()
  model.add(Flatten(input_shape=train_data.shape[1:]))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  H = model.fit(train_data, train_labels, epochs=epochs, 
            batch_size=batch_size, 
            validation_data=(validation_data, validation_labels))
  
  model.save(os.path.join(modelPath,'first_try_pretrained.h5'))

In [23]:
train_top_model()

Train on 1106 samples, validate on 294 samples
Epoch 1/50
1106/1106 [==============================] - 2s 2ms/step - loss: 0.6984 - accuracy: 0.7667 - val_loss: 0.2907 - val_accuracy: 0.8571
Epoch 2/50
1106/1106 [==============================] - 2s 1ms/step - loss: 0.4114 - accuracy: 0.8400 - val_loss: 0.2723 - val_accuracy: 0.8878
Epoch 3/50
1106/1106 [==============================] - 2s 2ms/step - loss: 0.3176 - accuracy: 0.8716 - val_loss: 0.6153 - val_accuracy: 0.8095
Epoch 4/50
1106/1106 [==============================] - 2s 1ms/step - loss: 0.2999 - accuracy: 0.8978 - val_loss: 0.3031 - val_accuracy: 0.8980
Epoch 5/50
1106/1106 [==============================] - 2s 1ms/step - loss: 0.2247 - accuracy: 0.9150 - val_loss: 0.3200 - val_accuracy: 0.8912
Epoch 6/50
1106/1106 [==============================] - 2s 2ms/step - loss: 0.1826 - accuracy: 0.9358 - val_loss: 0.3120 - val_accuracy: 0.9048
Epoch 7/50
1106/1106 [==============================] - 2s 1ms/step - loss: 0.1705 - accu

In [ ]:
print(H.history.keys())
#ploting the accuracy
plt.plot(H.history('accuracy'))
plt.plot(H.history('val_accuracy'))
plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()
#plotting the loss
plt.plot(H.history('loss'))
plt.plot(H.history('val_loss'))
plt.title('Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()